In [1]:
def link_circle_up(x, y, r, ax, finish=0):
    """
    Given two points, draw circle at the first point and link it to the second point
    without drawing the second point by default (so that it can repeat to build a long thread of bids).
    for the last point, pass the radius of the last circle to the argument 'finish'
    
    For example,

    fig = plt.figure()
    ax = fig.add_subplot(111)
    xpos = [1,1] &  ypos = [2,4]
    link_circle(xpos, ypos, 10, ax)

    xpos = [1,2] & ypos = [4,6]
    link_circle(xpos, ypos, 30, ax, finish=30)
    fig.show()
    """
    ax.plot(x[0], y[0], 'o', ms=r, lw=2,  alpha=0.7, mfc='orange')
    ax.plot(x, y, '-', c='black',alpha=0.7)
    if finish > 0:
        ax.plot(x[1], y[1], 'o', ms=20, lw=2, alpha=0.7, mfc='orange')    

def get_xarr(n):
    import numpy as np
    arr=[]
    a=0
    for i in range(n):
        a += (-1)**i * i
        arr.append(a)
    return np.asarray(arr)  
        
def recursive_tree(idx, tt, nstep, ax, x0, y0, dx, mass_unit=1e10):
    import tree.draw_merger_tree as dmt
    prgs = ctu.get_progenitors(tt, idx)
    i_this_gal = np.where(tt['id'] == idx)
    m = np.sqrt(tt[i_this_gal]["mvir"] / mass_unit)
    #print("IDX:", idx, "prgs: ",prgs, "mass:", m, i_this_gal)
    nprg = len(prgs)
    if nstep == 0:
        return 
    else:
        if nprg == 0:
            return
        else:
            if nprg > 1:
                #dx *= 1.1
                dx += 0.5
#                print("Branch!", nprg)

            #xarr = get_xarr(nprg) * dx + x0
            xarr = np.arange(nprg) * dx + x0
            for i, x in zip(prgs, xarr):
                link_circle_up([x0, x], [y0, y0 + 1], m, ax)
                recursive_tree(i, tt, nstep - 1, ax, x, y0 + 1, dx, mass_unit=mass_unit)


def extract_main_tree(treedata, idx=None, verbose=False):
    """
        Returns a single branch/trunk of tree following only the main progenitors.
        Works with both alltrees or atree.
        Search until no progenitor is found. Doesn't matter how long the given tree is. 
        Only earlier snapshots are searched for.
    """
    
    if idx == None:
        idx = treedata['id'][0]
        if verbose:
            print("No idx is given")
            print("idx = ", idx)

    
    nprg = 1
    ind_list=[np.where(treedata['id'] == idx)[0][0]]
    
    # main progenitor = mmp.
    while nprg > 0:        
        idx = ctu.get_progenitors(treedata, idx, main=True)
#        print(idx)
        ind_list.append(np.where(treedata['id'] == idx[0])[0][0])

        nprg = ctu.get_npr(treedata, idx[0])

    return treedata[ind_list]



def plot_atree(atree, galid):
    fig, ax = plt.subplots(1)
    ax.scatter(atree['aexp'], np.log10(atree['m']))
    ax.title(galid)
    plt.savefig(wdir + "mergertrees/" + sidgal + '.png')

###############################################################################
import matplotlib.pyplot as plt
from tree import treemodule
from tree import treeutils
import pickle
import numpy as np

In [2]:
import tree.ctutils as ctu
wdir='./'
alltrees = treemodule.load_tree(wdir, is_gal=True, load_ascii=True, no_dump=True)

Go
./GalaxyMaker/Trees/tree_0_0_0.dat
header
DAFFFS
Loading Consistent Tree data from ASCII is done
done
------ NOUT fixed
------ tree data extended


In [ ]:
aaa= ctu.augment_tree(alltrees.data, wdir, is_gal=True)

In [ ]:
nout_fi = 187
nout_ini = 30


i_final = np.where(alltrees.data["nout"] == nout_fi)
ttt_sub = alltrees.data[i_final]

nouts = np.arange(nout_fi - nout_ini + 1)

final_gals = ttt_sub['id']
final_gals_org = ttt_sub['Orig_halo_id']

plt.ioff()

#figure(figsize=[6,6])
#ax = fig.add_subplot(211)

#aexps = np.unique(alltrees.data["aexp"])[:len(nouts)]
aexps = np.unique(alltrees.data["aexp"])[:-len(nouts):-1]
zreds = ["%.2f" % (1/i -1) for i in aexps]

In [1]:
def _load_ascii(self, filename):
    cnt_header = 0

    datatype =[ 'f8','i8','i8','i8','i8','i8','i8','i8','i8','f8'\
               ,'f8','f8','i8','f8','f8','f8','i8','f8','f8','f8'\
               ,'f8','f8','f8','f8','f8','f8','f8','i8','i8','i8'\
               ,'i8','i8','i8','i8','i8','f8','i8']#,\      

    with open(filename, 'rb') as f:   
        for i in range(180):
            line = f.readline()
            line = line.decode('utf-8')
            if line[0] != '#':
                self.ntrees = int(line) # The first line after the header is the number of trees.
                cnt_header = f.tell()
                break
        f.seek(cnt_header)
        self.data = np.genfromtxt(f,dtype=datatype)

    self.data.dtype.names=(\
        'aexp','id','desc_aexp','desc_id','nprog','pid','upid','desc_pid','phantom','sam_mvir'\
        ,'mvir','rvir','rs','vrms','mmp','aexp_last_MM','vmax','x','y','z'\
        ,'vx','vy','vz','jx','jy','jz','spin','b_id','d_id','tree_root_id'\
        ,'Orig_halo_id','nout','next_coprogenitor_d_id','last_progenitor_d_id'\
        ,'last_mainleaf_depthfirst_id', 'tidal_force', 'tidal_id')

    print("Loading Consistent Tree data from ASCII is done")

In [5]:
import numpy as np

class abc():
    pass

ccc = treemodule.CTree()

ccc._load_ascii('./GalaxyMaker/Trees/tree_0_0_0.dat')

./GalaxyMaker/Trees/tree_0_0_0.dat
header
DAFFFS
Loading Consistent Tree data from ASCII is done


### ###


In [8]:
atree = ctu.extract_a_tree(alltrees.data, 270444)

In [5]:
indgal = (atree['nout'] > 87) * (atree['nout'] < 105) * (atree['m'] > 1e10)

In [5]:
print(atree['id'][indgal])

[172573 171034 171035 169490 169491 167950 167951 166399 166400 164832
 164833 163270 161715 160144 160145 158542 158543 156939 156940 155312
 155314 153694 153696 152060 152062 150407 150409 148739 148741 147040
 147042]


In [6]:
m1 = extract_main_tree(alltrees.data, idx=171034)
m2 = extract_main_tree(alltrees.data, idx=171035)

In [9]:
sub1 = ctu.extract_main_tree(atree, idx=171034, no_subset=True)
sub2 = ctu.extract_main_tree(atree, idx=171035, no_subset=True)

In [10]:
plt.scatter(sub1['aexp'], sub1['m'], c='red')
plt.scatter(sub2['aexp'], sub2['m'], c='blue')
plt.show()

In [13]:
sub1.dtype

dtype([('aexp', '<f8'), ('id', '<i8'), ('desc_aexp', '<i8'), ('desc_id', '<i8'), ('nprog', '<i8'), ('pid', '<i8'), ('upid', '<i8'), ('desc_pid', '<i8'), ('phantom', '<i8'), ('sam_mvir', '<f8'), ('mvir', '<f8'), ('rvir', '<f8'), ('rs', '<i8'), ('vrms', '<f8'), ('mmp', '<f8'), ('aexp_last_MM', '<f8'), ('vmax', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<i8'), ('jx', '<i8'), ('jy', '<i8'), ('jz', '<f8'), ('spin', '<i8'), ('b_id', '<i8'), ('d_id', '<i8'), ('tree_root_id', '<i8'), ('Orig_halo_id', '<i8'), ('nout', '<i8'), ('next_coprogenitor_d_id', '<i8'), ('last_progenitor_d_id', '<f8'), ('np', '<i4'), ('m', '<f4'), ('r', '<f4'), ('tvir', '<f4'), ('cvel', '<f4'), ('sig', '<f4'), ('sigbulge', '<f4'), ('mbulge', '<f4')])

In [11]:
qlist = ["mvir", "rvir", "rs", "vrms", "vmax",
         "x", "y", "z", "vx", "vy",
         "vz", "jx", "jy", "jz", "spin", "np", "m", "r",
         "tvir", "cvel", "sig", "sigbulge", "mbulge"][:]
fig, ax = plt.subplots(8,3)
ax= ax.ravel()

for i, q in enumerate(qlist):
    ax[i].plot(sub1['aexp'], sub1[q], c='blue')
    ax[i].plot(sub2['aexp'], sub2[q], c='red')
    ax[i].set_title(q)

plt.tight_layout()
plt.show()

In [7]:
ind1 = m1['nout'] > 80
ind2 = m2['nout'] > 80
plt.plot(m1['nout'][ind1],m1['m'][ind1])
plt.plot(m2['nout'][ind2],m2['m'][ind2])
plt.show()

In [8]:
plt.scatter(atree[indgal]['aexp'], atree[indgal]['m'])
plt.show()

In [46]:
print(atree.dtype)
print(atree[0])

[('aexp', '<f8'), ('id', '<i8'), ('desc_aexp', '<i8'), ('desc_id', '<i8'), ('nprog', '<i8'), ('pid', '<i8'), ('upid', '<i8'), ('desc_pid', '<i8'), ('phantom', '<i8'), ('sam_mvir', '<f8'), ('mvir', '<f8'), ('rvir', '<f8'), ('rs', '<i8'), ('vrms', '<f8'), ('mmp', '<f8'), ('aexp_last_MM', '<f8'), ('vmax', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<i8'), ('jx', '<i8'), ('jy', '<i8'), ('jz', '<f8'), ('spin', '<i8'), ('b_id', '<i8'), ('d_id', '<i8'), ('tree_root_id', '<i8'), ('Orig_halo_id', '<i8'), ('nout', '<i8'), ('next_coprogenitor_d_id', '<i8'), ('last_progenitor_d_id', '<f8'), ('np', '<i4'), ('m', '<f4'), ('r', '<f4'), ('tvir', '<f4'), ('cvel', '<f4'), ('sig', '<f4'), ('sigbulge', '<f4'), ('mbulge', '<f4')]
(1.0, 270444, -1, -1, 1, -1, -1, -1, 0, 128133000000.0, 126100000000.0, 318.616, -1, 0.0, 1.0, 0.95, -1, 108.567, 160.908, 76.7034, -358.22, -467.53, -1, -1, -1, 913300000000000.0, -1, 38691, 38691, 270444, 4, 187, -1, 40200.0, 187238, 8

In [75]:
for t,q in zip(atree.dtype.names, atree[0]):
    if isinstance(q, np.int64):
        #print("int")
        print(" {}  : {:d}".format(t, q))
    elif q > 1e4:
        print(" {}  : {:.3e}".format(t, q))
    else:
        print(" {}  : {:.2f}".format(t, q))

 aexp  : 1.00
 id  : 270444
 desc_aexp  : -1
 desc_id  : -1
 nprog  : 1
 pid  : -1
 upid  : -1
 desc_pid  : -1
 phantom  : 0
 sam_mvir  : 1.281e+11
 mvir  : 1.261e+11
 rvir  : 318.62
 rs  : -1
 vrms  : 0.00
 mmp  : 1.00
 aexp_last_MM  : 0.95
 vmax  : -1
 x  : 108.57
 y  : 160.91
 z  : 76.70
 vx  : -358.22
 vy  : -467.53
 vz  : -1
 jx  : -1
 jy  : -1
 jz  : 9.133e+14
 spin  : -1
 b_id  : 38691
 d_id  : 38691
 tree_root_id  : 270444
 Orig_halo_id  : 4
 nout  : 187
 next_coprogenitor_d_id  : -1
 last_progenitor_d_id  : 4.020e+04
 np  : 1.872e+05
 m  : 8.878e+10
 r  : 0.00
 tvir  : 1.015e+05
 cvel  : 53.17
 sig  : 102.47
 sigbulge  : 92.47
 mbulge  : 0.44


In [ ]:
import os
if not os.path.isdir(wdir + "mergertrees/"):
    os.mkdir(wdir + "mergertrees/")


fig, ax = plt.subplots(1,2)
fig.set_size_inches([12,6])
for galid in final_gals:
    sidgal = str(galid).zfill(5)      
    
    #print(zreds)
    atree = ctu.extract_a_tree(alltrees.data, galid)
    mtree = extract_main_tree(atree)
    
    ax[0].scatter(atree['aexp'], np.log10(atree['m']), edgecolors='none', alpha=0.3)
    ax[0].scatter(mtree['aexp'], np.log10(mtree['m']), edgecolors='none', alpha=0.6,
                  facecolors='red')
    ax[0].set_xlim([0.15,1.1])
    ax[0].set_xticks(aexps[0:151:20])
    ax[0].set_xticklabels(zreds[0:151:20])
    ax[0].set_title(galid)
    
    recursive_tree(galid, atree, 150, ax[1], 0, 0, 0.8, mass_unit=2e8)
    
    # y axis label (redshift)
    ax[1].set_ylabel("Redshift")
    ax[1].set_ylim([-5,155])
    ax[1].set_yticks(range(0,151,10))
    ax[1].set_yticklabels(zreds[0:151:10])
    #plt.yticks(range(0,151,10), zreds[0:151:10])
    ax[1].set_title(sidgal + ", " + str(atree[0]['Orig_halo_id']))
    #fig.show()
    plt.savefig(wdir + "mergertrees/" + sidgal + '.png')
    ax[0].clear()
    ax[1].clear()
    
    #plt.close()


plt.close()

In [34]:
import tree.halomodulemodule as hmo

In [37]:
gg = hmo.Halo(nout=187, is_gal=True, load=True)

In [39]:
gg.data.dtype

dtype((numpy.record, [('np', '<i4'), ('id', '<i4'), ('level', '<i4'), ('host', '<i4'), ('sub', '<i4'), ('nsub', '<i4'), ('nextsub', '<i4'), ('m', '<f4'), ('mvir', '<f4'), ('r', '<f4'), ('rvir', '<f4'), ('tvir', '<f4'), ('cvel', '<f4'), ('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('vx', '<f4'), ('vy', '<f4'), ('vz', '<f4'), ('ax', '<f4'), ('ay', '<f4'), ('az', '<f4'), ('sp', '<f4'), ('idx', '<i4'), ('p_rho', '<f4'), ('p_c', '<f4'), ('energy', '<f8', (3,)), ('radius', '<f8', (4,)), ('sig', '<f4'), ('sigbulge', '<f4'), ('mbulge', '<f4'), ('hosthalo', '<i4'), ('g_nbin', '<i4'), ('g_rr', '<f4', (100,)), ('g_rho', '<f4', (100,))]))

In [44]:
gg.data['az']

array([ -1.13061008e+01,  -7.01060100e-03,   9.38960731e-01,
         4.52640676e+00,  -3.07331115e-01,  -2.36604512e-02,
         1.46900741e-02,  -8.77192542e-02,  -4.45459175e+00,
        -1.81807458e-01,   3.49070072e+00,  -2.82486659e-02,
        -7.14577660e-02,  -1.05326967e-02,   2.70612895e-01,
         1.45428600e-02,  -5.92599273e-01,  -2.03332162e+00,
        -1.08956069e-01,   9.83856869e+00,  -2.38497835e-02,
         2.58718014e-01,   1.66282490e-01,  -1.70101392e+00,
        -7.14281201e-02,   1.19412087e-01,  -6.33126032e-03,
        -5.22932751e-05,  -1.20246212e-03,   7.46302074e-03,
         2.38446984e-02,   1.02446623e-01,   2.28215218e-01,
        -4.77806330e-02,   1.25209346e-01,   2.12405226e-03,
        -9.21617672e-02,  -2.27775681e-03,   1.91194534e+00,
        -6.06292009e-01,   3.07610491e-03,   1.74930636e-02,
        -1.18322065e-02,   3.23338807e-03,  -7.33600976e-03,
         1.71045613e-04,   1.11963636e-04,   4.28991541e-02,
         2.71807648e-02,